<a href="https://colab.research.google.com/github/sush7101/Data_Science/blob/main/DATA_AUGMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
!pip install sdv
import os
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.sampling import Condition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import cleaned data
data=pd.read_csv(r'/content/drive/MyDrive/ANOMALY_DETECTION/clean_data.csv')

In [ ]:
# Convert categorical columns to string type
cat_cols=['Fwd PSH Flags', 'Bwd PSH Flags','Fwd URG Flags', 'Bwd URG Flags','FIN Flag Count','SYN Flag Count','RST Flag Count','PSH Flag Count','ACK Flag Count','URG Flag Count','CWE Flag Count','ECE Flag Count','Label','Destination_port_group']
data[cat_cols] = data[cat_cols].astype('object')

In [ ]:
data['Label'].value_counts()

,count
Label,
BENIGN,2271285
DoS Hulk,230124
PortScan,158804
DDoS,128025
DoS GoldenEye,10293
FTP-Patator,7935
SSH-Patator,5897
DoS slowloris,5796
DoS Slowhttptest,5499


In [ ]:
benign_total=(data['Label']=='BENIGN').sum()
attack_total=(data['Label']!='BENIGN').sum()
print(f"Total benign: {benign_total}")
print(f"Total attacks: {attack_total}")

Total benign: 2271285
Total attacks: 556556


In [ ]:
#Undersample 'BENIGN'to the highest attack('DoS Hulk') to reduce redundancy and faster prepocessing
undersample_size = 230124
#Create "training_data" that maintains a balance between well represented and under represented data
benign_data = data[data['Label'] == 'BENIGN'].sample(n=undersample_size, random_state=42)
attack_data = data[data['Label'] != 'BENIGN']
training_data = pd.concat([benign_data, attack_data]).sample(frac=1, random_state=42).reset_index(drop=True)
# Save the undersampled training data to your Google Drive
training_data.to_csv('/content/drive/MyDrive/ANOMALY_DETECTION/undersampled_data.csv', index=False)
print("Undersampled data saved to Google Drive.")

Undersampled data saved to Google Drive.


In [ ]:
import os
# Mount Google Drive to save data and model checkpoints
drive.mount('/content/drive')

# Define the directory for your checkpoints
checkpoint_dir = '/content/drive/MyDrive/ANOMALY_DETECTION/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Step 3: Create metadata from the new, smaller dataset
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(training_data)

total_epochs = 100
print(f"\nStarting CTGAN training for {total_epochs} epochs...")

ctgan = CTGANSynthesizer(metadata, epochs=total_epochs, verbose=True, cuda=True)

# The fit() method will now run for the entire 300 epochs
ctgan.fit(training_data)

# Save the final model after all 300 epochs are complete
final_model_filename = os.path.join(checkpoint_dir, 'ctgan_final_model.pkl')
ctgan.save(final_model_filename)
print("Final CTGAN training complete and model saved.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Starting CTGAN training for 100 epochs...


/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:167: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:133: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sdv/single_table/ctgan.py:252: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  num_categories = data[column].fillna(np.nan).nunique(dropna=False)
Gen. (-1.20) | Discrim. (0.16): 100%|██████████| 100/100 [3:55:35<00:00, 141.36s/it]


Final CTGAN training complete and model saved.


In [ ]:

# Load trained CTGAN model
ctgan = CTGANSynthesizer.load('/content/drive/MyDrive/ANOMALY_DETECTION/ctgan_final_model.pkl')